# Importando as bibliotecas necessárias

In [1]:
import cv2
import os
import glob
import numpy as np

# Obtenção das fotos do tabuleiro para calibração das cameras e consequentemente obtenção dos POI

In [2]:
capture_key = ord('c')  

camera_indices = [1, 2]

image_dirs = ['camera1_images', 'camera2_images']
for dir_name in image_dirs:
    os.makedirs(dir_name, exist_ok=True)
    os.makedirs(dir_name + '_calibrated', exist_ok=True)

#inicializar as cameras
cameras = []
for index in camera_indices:
    camera = cv2.VideoCapture(index)
    cameras.append(camera)

counter = 0
# Comecar a captura
while True:
    #ler os frames de cada camera
    frames = []
    for camera in cameras:
        ret, frame = camera.read()
        frames.append(frame)

    for i, frame in enumerate(frames):
        cv2.imshow(f'Camera {i+1}', frame)

    #checar se a tecla de captura foi apertada
    key = cv2.waitKey(1)
    if key == capture_key:
        counter+=1
        for i, frame in enumerate(frames):
            image_path = os.path.join(image_dirs[i], f'image_{i+1}_{counter}.jpg')
            cv2.imwrite(image_path, frame)
            print(f'Image saved: {image_path}')
    #Checar se a tecla de saida esta pressionada
    if key == ord('q'):
        break

for camera in cameras:
    camera.release()
cv2.destroyAllWindows()

In [3]:
def same_pict():
    capture_key = ord('c')  

    camera_indices = [1, 2]


    image_dirs = 'camera_1_2_frames'
    os.makedirs(image_dirs, exist_ok=True)

    cameras = []
    for index in camera_indices:
        camera = cv2.VideoCapture(index)
        cameras.append(camera)

    counter = 0
    while True:
        frames = []
        for camera in cameras:
            ret, frame = camera.read()
            frames.append(frame)

        for i, frame in enumerate(frames):
            cv2.imshow(f'Camera {i+1}', frame)

        key = cv2.waitKey(1)
        if key == capture_key:
            counter+=1
            for i, frame in enumerate(frames):
                image_path = os.path.join(image_dirs, f'image_{i+1}_{counter}.jpg')
                cv2.imwrite(image_path, frame)
                print(f'Image saved: {image_path}')
        if key == ord('q'):
            break

    for camera in cameras:
        camera.release()
    cv2.destroyAllWindows()
same_pict()

# Obtenção dos POI

### Definição de funções aux

In [4]:
def draw_chess_write(camera):
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
  world_scaling = 2.8
  objp = np.zeros((6*8,3), np.float32)
  objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)
  objp = world_scaling* objp
  objpoints = [] 
  imgpoints = [] 
  images = glob.glob(f'./camera{camera}_images/*.jpg')
  i=1
  for fname in images:
    
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)
    if ret == True:
        name = i
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        cv2.drawChessboardCorners(img, (8,6), corners2, ret)
        cv2.imshow('img',img)
        cv2.imwrite(f'./camera{camera}_images_calibrated/calib_' + str(name) + '_calib.jpg',img)
        cv2.waitKey(500)
        i+=1
        
  cv2.destroyAllWindows()
  return objpoints, imgpoints, gray

def stereo_calibrate(mtx0, dist0, mtx1, dist1, frames_prefix_c0, frames_prefix_c1):
    c0_images_names = sorted(glob.glob(frames_prefix_c0))
    c1_images_names = sorted(glob.glob(frames_prefix_c1))

    c0_images = [cv2.imread(imname, 1) for imname in c0_images_names]
    c1_images = [cv2.imread(imname, 1) for imname in c1_images_names]

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)

    rows = 8
    columns = 6
    world_scaling = 2.8

    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp

    width = c0_images[0].shape[1]
    height = c0_images[0].shape[0]

    imgpoints_left = [] # 2d points in image plane.
    imgpoints_right = []

    objpoints = [] # 3d point in real world space

    for frame0, frame1 in zip(c0_images, c1_images):
        gray1 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        c_ret1, corners1 = cv2.findChessboardCorners(gray1, (rows, columns), None)
        c_ret2, corners2 = cv2.findChessboardCorners(gray2, (rows, columns), None)

        if c_ret1 == True and c_ret2 == True:

            corners1 = cv2.cornerSubPix(gray1, corners1, (11, 11), (-1, -1), criteria)
            corners2 = cv2.cornerSubPix(gray2, corners2, (11, 11), (-1, -1), criteria)

            p0_c1 = corners1[0,0].astype(np.int32)
            p0_c2 = corners2[0,0].astype(np.int32)

            cv2.putText(frame0, 'O', (p0_c1[0], p0_c1[1]), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 1)
            cv2.drawChessboardCorners(frame0, (rows,columns), corners1, c_ret1)
            cv2.imshow('img', frame0)

            cv2.putText(frame1, 'O', (p0_c2[0], p0_c2[1]), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 1)
            cv2.drawChessboardCorners(frame1, (rows,columns), corners2, c_ret2)
            cv2.imshow('img2', frame1)
            k = cv2.waitKey(0)

            if k & 0xFF == ord('s'):
                print('skipping')
                continue

            objpoints.append(objp)
            imgpoints_left.append(corners1)
            imgpoints_right.append(corners2)

    stereocalibration_flags = cv2.CALIB_FIX_INTRINSIC
    ret, CM1, dist0, CM2, dist1, R, T, E, F = cv2.stereoCalibrate(objpoints, imgpoints_left, imgpoints_right, mtx0, dist0,
                                                                 mtx1, dist1, (width, height), criteria = criteria, flags = stereocalibration_flags)

    cv2.destroyAllWindows()
    return R, T

def get_poi(mtx_1):
    fx = mtx_1[0][0]
    fy = mtx_1[1][1]
    cx = mtx_1[0][2]
    cy = mtx_1[1][2]
    return fx,fy,cx,cy

In [5]:
objpoints_1,imgpoints_1,gray_1 = draw_chess_write('1')
objpoints_2,imgpoints_2,gray_2 = draw_chess_write('2')
ret_1, mtx_1, dist_1, rvecs_1, tvecs_1 = cv2.calibrateCamera(objpoints_1, imgpoints_1, gray_1.shape[::-1], None, None)
ret_2, mtx_2, dist_2, rvecs_2, tvecs_2 = cv2.calibrateCamera(objpoints_2, imgpoints_2, gray_2.shape[::-1], None, None)

In [6]:
frames_prefix_c0 = os.path.join('camera_1_2_frames','image_1*')
frames_prefix_c1 = os.path.join('camera_1_2_frames','image_2*')

In [7]:
poi_1 = get_poi(mtx_1)
poi_2 = get_poi(mtx_2)

In [8]:
poi_1

(999.6184685265891, 997.587932339299, 217.2324828046475, 223.21015049905702)

In [9]:
poi_2

(1007.8357024349465, 1005.46347350523, 272.03062315393083, 254.10464615971662)

# Obtenção dos parametros de montagem

In [10]:
R,T = stereo_calibrate(mtx_1,dist_1,mtx_2,dist_2,frames_prefix_c0,frames_prefix_c1)

In [11]:
width = cv2.imread('./camera1_images/image_1_1.jpg').shape[1]
height = cv2.imread('./camera1_images/image_1_1.jpg').shape[0]

In [12]:
R_vec = R
T_vec = T

# Escrevendo os resultados

In [ ]:
arquivo = open("dados_retirada.txt",'w')
arquivo.write('POI = [fx,fy,cx,cy]')
arquivo.write('\n')
arquivo.write('POI_1 = ' + str(poi_1) +'\n')
arquivo.write('POI_2 = ' + str(poi_2) +'\n')
arquivo.write('POR:' +'\n'+ 'matriz_rotacao = ' + str(R_vec) +'\n'+'matriz transalacao = ' + str(T_vec))
arquivo.close()

# Debug

In [ ]:
print(R)

In [ ]:
print( T )

In [ ]:
print(mtx_1)

In [ ]:
print(dist_1)

In [ ]:
print(mtx_2)

In [ ]:
print(dist_2)

In [ ]:
teste = cv2.imread('./camera1_images/image_1_1.jpg')

In [ ]:
teste